In [1]:
%pip install --upgrade --quiet python-dotenv langchain langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 9.0 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")


Enter your Groq API Key: ··········


In [3]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0
)


In [4]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

prompt_standard = f"Answer this question: {question}"

print("--- STANDARD ---")
print(llm.invoke(prompt_standard).content)


--- STANDARD ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


In [5]:
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- CHAIN OF THOUGHT ---")
print(llm.invoke(prompt_cot).content)


--- CHAIN OF THOUGHT ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the new tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


In [6]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7
)


In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give one unique creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist.
    Pick the most sustainable solution (not bribery).
    Explain why.
    """
)

tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- TREE OF THOUGHTS ---")
print(tot_chain.invoke(problem))


--- TREE OF THOUGHTS ---
As a child psychologist, I would recommend **Solution 1: "Veggie Faces"** and **Solution 3: Create a "Veggie Face" on Their Plate** as the most sustainable solutions, but with some slight modifications to ensure they align with healthy eating habits.

**Why "Veggie Faces" and "Veggie Face" are the most sustainable solutions:**

1. **Engagement:** Both solutions encourage engagement and creativity, which are essential for children to develop a positive relationship with food.
2. **Positive associations:** By making mealtime more fun and interactive, children are more likely to develop positive associations with vegetables and other healthy foods.
3. **Gradual exposure:** Both solutions allow for gradual exposure to new vegetables, which is essential for introducing children to new foods.
4. **No bribery:** Unlike some other solutions that involve offering rewards or treats for eating vegetables, "Veggie Faces" and "Veggie Face" are not bribing children to eat ce

In [8]:
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)


In [9]:
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for '{topic}':

    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Combine the TECHNOLOGY of Sci-Fi,
    the PASSION of Romance,
    and the FEAR of Horror.

    Write one paragraph.
    """
)

got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- GRAPH OF THOUGHTS ---")
print(got_chain.invoke("Time Travel"))


--- GRAPH OF THOUGHTS ---
As Emily, a brilliant and daring physicist, travels back to the 1920s in her time machine, she's immediately captivated by the charming soldier, James, who sweeps her off her feet with his dashing smile and romantic gestures. But as they fall deeply in love, Emily begins to notice strange occurrences that hint at a dark presence lurking in the shadows - a malevolent entity from the past that has been unleashed by her time travel, and is now stalking them relentlessly, determined to claim their souls. As Emily and James fight to survive the night and prevent a catastrophic future, they must also confront the devastating consequences of changing the past, and the ultimate question: can their love be strong enough to overcome the horrors that threaten to tear them apart, or will the very fabric of time tear them asunder?
